In [1]:
# Load the SQL extension first (only shows once)
%load_ext sql

# Import required libraries
import os
from dotenv import load_dotenv
import urllib

# Load variables from your .env file (with UTF-8 encoding)
load_dotenv(encoding='utf-8-sig')

# Fetch variables
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver = os.getenv("DRIVER")

# Build the connection string
params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
)
connection_string = f"mssql+pyodbc:///?odbc_connect={params}"

%sql {connection_string}
%config SqlMagic.displaycon = False  # Hide connection info
print("Secure connection established successfully!")

Secure connection established successfully!


CTE Course: Lesson 1 — Basic CTE Usage

Selecting Basic Employee Information

If you just want to select the first name, last name, and ID from the Person.Person table, you can do it with a simple SELECT statement:

In [3]:
%%sql

SELECT TOP 10 -- remove TOP 10 to see all results
    PEP.[FirstName],
    PEP.[LastName],
    PEP.BusinessEntityID
FROM Person.Person PEP;

Done.


FirstName,LastName,BusinessEntityID
Syed,Abbas,285
Catherine,Abel,293
Kim,Abercrombie,295
Kim,Abercrombie,2170
Kim,Abercrombie,38
Hazem,Abolrous,211
Sam,Abolrous,2357
Humberto,Acevedo,297
Gustavo,Achong,291
Pilar,Ackerman,299


This query will return all people in the Person.Person table with their first name, last name, and BusinessEntityID.

Using a CTE (Common Table Expression)

A CTE allows you to define a temporary result set that you can reference in a SELECT, INSERT, UPDATE, or DELETE statement. It’s especially useful when you want to simplify complex queries or break them into readable steps.

For example, if you want to combine person information with job titles from HumanResources.Employee, you can do:

In [24]:
%%sql

WITH EmployeeDetails AS (
    SELECT 
        HRE.[BusinessEntityID],
        HRE.[JobTitle]
    FROM HumanResources.Employee HRE
)
SELECT TOP 10
    PEP.[FirstName] /*remove TOP 10 to see all results*/,
    PEP.[LastName],
    ED.BusinessEntityID,
    ED.JobTitle
FROM Person.Person PEP
JOIN EmployeeDetails AS ED 
    ON PEP.BusinessEntityID = ED.BusinessEntityID
ORDER BY ED.BusinessEntityID;

Done.


FirstName,LastName,BusinessEntityID,JobTitle
Ken,Sánchez,1,Chief Executive Officer
Terri,Duffy,2,Vice President of Engineering
Roberto,Tamburello,3,Engineering Manager
Rob,Walters,4,Senior Tool Designer
Gail,Erickson,5,Design Engineer
Jossef,Goldberg,6,Design Engineer
Dylan,Miller,7,Research and Development Manager
Diane,Margheim,8,Research and Development Engineer
Gigi,Matthew,9,Research and Development Engineer
Michael,Raheem,10,Research and Development Manager


TOPIC: CTE — Filtering by Value

Question:
List all products that have a ListPrice greater than 100.

Tables: Production.Product
Concept: Filtering rows

Simple (Non-CTE) Approach

The easiest way to do it is with a standard SELECT and WHERE clause

In [26]:
%%sql

SELECT TOP 10  /*remove TOP 10 to see all results*/
	[ProductID], [ListPrice]
FROM Production.Product
WHERE [ListPrice] > 100

Done.


ProductID,ListPrice
514,133.3400
515,147.1400
516,196.9200
517,133.3400
518,147.1400
519,196.9200
520,133.3400
521,147.1400
522,196.9200
680,1431.5000


Using a CTE

If you want to use a CTE to structure your query more clearly or prepare data for additional steps, you can write:

In [27]:
%%sql

WITH ListPriceabove100 AS(
SELECT
	[ProductID] Products,
	[Name] ProductName,
	[MakeFlag] Flag,
	[ListPrice] Price
FROM Production.Product
WHERE [ListPrice] > 100)
SELECT	TOP 10 /*remove TOP 10 to see all results*/
	Products,
	Price
FROM ListPriceabove100

Done.


Products,Price
514,133.3400
515,147.1400
516,196.9200
517,133.3400
518,147.1400
519,196.9200
520,133.3400
521,147.1400
522,196.9200
680,1431.5000


TOPIC: CTE — Comparing to an Aggregate

Sometimes you need to compare each row in a table to an aggregate value such as an average. In this example, we want to find all sales orders where the OrderQty is greater than the average OrderQty of all orders.

This is a perfect example to show the link between subqueries and CTEs:
If you understand subqueries, then you already understand the logic behind CTEs — you’re just moving that logic to the top of the query.

Approach 1: Using a Subquery

A classic subquery returns the average OrderQty, and we compare each row to it:

In [29]:
%%sql

SELECT TOP 10 /*remove TOP 10 to see all results*/
    SOD.[SalesOrderID],
    SOD.[OrderQty]
FROM Sales.SalesOrderDetail SOD
WHERE SOD.[OrderQty] > 
    (
        SELECT
            AVG(SOD2.[OrderQty])
        FROM Sales.SalesOrderDetail SOD2
    );

Done.


SalesOrderID,OrderQty
43659,3
43659,3
43659,6
43659,4
43661,4
43661,4
43661,3
43661,4
43661,5
43662,3


What this does:

The inner query calculates the average quantity of all orders.

The outer query returns only the orders where OrderQty is higher than that average.

Approach 2: Using a CTE

A CTE can be used to compute the same aggregate before the main query runs.

In [31]:
%%sql

WITH AverageOrderQuantity AS (
    SELECT 
        AVG(SOD.[OrderQty]) AS AverageOrder
    FROM Sales.SalesOrderDetail SOD
)
SELECT TOP 10 /*remove TOP 10 to see all results*/
    SOD2.[SalesOrderID],
    SOD2.[OrderQty]
FROM Sales.SalesOrderDetail SOD2
CROSS JOIN AverageOrderQuantity AOQ
WHERE SOD2.[OrderQty] > AOQ.AverageOrder;


Done.


SalesOrderID,OrderQty
43659,3
43659,3
43659,6
43659,4
43661,4
43661,4
43661,3
43661,4
43661,5
43662,3


Explanation:

The CTE AverageOrderQuantity calculates the average order quantity once.

The CROSS JOIN attaches that single value to every row.

The WHERE clause filters orders above the average.

TOPIC: CTE — Filtering by Existence

In this lesson, we want to display all customers who belong to a territory (i.e., TerritoryID is not null). We also want to show their first name, last name, and territory.

This demonstrates simple filtering, and again shows how a query can be written using:

A normal SELECT

A subquery

A CTE

Approach 1: Simple Filtering (No Subquery, No CTE)

In [42]:
%%sql 

SELECT TOP 10 /*remove TOP 10 to see all results*/
    PEP.[FirstName],
    PEP.[LastName],
    SC.[TerritoryID]
FROM Person.Person AS PEP
JOIN Sales.Customer SC
    ON PEP.BusinessEntityID = SC.PersonID
WHERE SC.[TerritoryID] IS NOT NULL
ORDER BY TerritoryID;

Done.


FirstName,LastName,TerritoryID
Lauren,Walker,1
Ian,Jenkins,1
Sydney,Bennett,1
Destiny,Wilson,1
Ethan,Zhang,1
Jesse,Murphy,1
Noah,Powell,1
Angela,Murphy,1
Grace,Butler,1
Caleb,Carter,1


What this does:

Joins people with the customer table

Filters customers to only those who belong to a territory

Returns name + territory

Approach 2: Using a CTE

Now we move the filtering step into a CTE:

In [43]:
%%sql

WITH TerritoryNotNull AS (
    SELECT
        [PersonID],
        [TerritoryID] AS Territories
    FROM Sales.Customer 
    WHERE TerritoryID IS NOT NULL
)
SELECT TOP 10 /*remove TOP 10 to see all results*/
    PEP.[FirstName],
    PEP.[LastName],
    TNN.Territories
FROM Person.Person PEP
JOIN TerritoryNotNull AS TNN
    ON PEP.BusinessEntityID = TNN.PersonID
ORDER BY TNN.Territories;


Done.


FirstName,LastName,Territories
Lauren,Walker,1
Ian,Jenkins,1
Sydney,Bennett,1
Destiny,Wilson,1
Ethan,Zhang,1
Jesse,Murphy,1
Noah,Powell,1
Angela,Murphy,1
Grace,Butler,1
Caleb,Carter,1


TOPIC: CTE — Latest Department per Employee

We want to find the most recent department each employee has worked in, along with their job title.
This requires identifying the latest StartDate per employee, which is a perfect case for ROW_NUMBER() in a CTE.

Tables:

HumanResources.Employee

HumanResources.EmployeeDepartmentHistory

HumanResources.Department

Concept: CTE with ROW_NUMBER() for latest entry


Approach 1: Subquery Version

In [46]:
%%sql

SELECT TOP 10  /*remove TOP 10 to see all results*/
    e.BusinessEntityID,
    e.JobTitle,
    d.Name AS DepartmentName,
    edh.StartDate AS DepartmentStartDate
FROM HumanResources.EmployeeDepartmentHistory edh
JOIN HumanResources.Employee e
    ON e.BusinessEntityID = edh.BusinessEntityID
JOIN HumanResources.Department d
    ON d.DepartmentID = edh.DepartmentID
WHERE edh.StartDate = (
    SELECT MAX(edh2.StartDate)
    FROM HumanResources.EmployeeDepartmentHistory edh2
    WHERE edh2.BusinessEntityID = edh.BusinessEntityID
)
ORDER BY edh.StartDate DESC;


Done.


BusinessEntityID,JobTitle,DepartmentName,DepartmentStartDate
234,Chief Financial Officer,Executive,2013-11-14
286,Sales Representative,Sales,2013-05-30
288,Sales Representative,Sales,2013-05-30
285,Pacific Sales Manager,Sales,2013-03-14
284,Sales Representative,Sales,2012-09-30
250,Purchasing Manager,Purchasing,2012-07-15
290,Sales Representative,Sales,2012-05-30
289,Sales Representative,Sales,2012-05-30
287,European Sales Manager,Sales,2012-04-16
224,Scheduling Assistant,Production Control,2011-09-01


What this does:

Finds the maximum (latest) StartDate for each employee.

Returns name, job title, department, and when they started in it.

Limited to TOP 10 rows for notebook readability.

Approach 2: CTE Version (Cleaner + More Scalable)

In [47]:
%%sql

WITH LatestDepartment AS (
    SELECT
        edh.BusinessEntityID,
        edh.DepartmentID,
        edh.StartDate,
        ROW_NUMBER() OVER (
            PARTITION BY edh.BusinessEntityID
            ORDER BY edh.StartDate DESC
        ) AS rn
    FROM HumanResources.EmployeeDepartmentHistory edh
)
SELECT TOP 10  /*remove TOP 10 to see all results*/
    e.BusinessEntityID,
    e.JobTitle,
    d.Name AS DepartmentName,
    ld.StartDate AS DepartmentStartDate
FROM LatestDepartment ld
JOIN HumanResources.Employee e
    ON e.BusinessEntityID = ld.BusinessEntityID
JOIN HumanResources.Department d
    ON d.DepartmentID = ld.DepartmentID
WHERE ld.rn = 1
ORDER BY ld.StartDate DESC;


Done.


BusinessEntityID,JobTitle,DepartmentName,DepartmentStartDate
234,Chief Financial Officer,Executive,2013-11-14
286,Sales Representative,Sales,2013-05-30
288,Sales Representative,Sales,2013-05-30
285,Pacific Sales Manager,Sales,2013-03-14
284,Sales Representative,Sales,2012-09-30
250,Purchasing Manager,Purchasing,2012-07-15
289,Sales Representative,Sales,2012-05-30
290,Sales Representative,Sales,2012-05-30
287,European Sales Manager,Sales,2012-04-16
224,Scheduling Assistant,Production Control,2011-09-01


Why this is better:

ROW_NUMBER() ranks departmental history per employee.

rn = 1 returns only the most recent row.

Much more efficient if employees have many history records.

TOPIC: CTE — Employee Pay History Trends

In this lesson, we want to find how employee pay rates have changed over time, and calculate the percentage increase or decrease between the most recent and previous pay rate.

Tables:

HumanResources.Employee

HumanResources.EmployeePayHistory

Concept: CTE with ROW_NUMBER() + self join to compare latest vs previous rate

In [48]:
%%sql

WITH RankedPaidHistory AS (
    SELECT
        [BusinessEntityID],
        [RateChangeDate],
        [Rate],
        ROW_NUMBER() OVER (
            PARTITION BY [BusinessEntityID]
            ORDER BY [RateChangeDate] DESC
        ) AS rn
    FROM HumanResources.EmployeePayHistory
)
SELECT TOP 10  /*remove TOP 10 to see all results*/
    HRE.[BusinessEntityID],
    HRE.[JobTitle],
    RLatest.Rate AS LatestRate,
    RPrevious.Rate AS PreviousRate,
    RLatest.RateChangeDate AS LatestChangeDate,
    RPrevious.RateChangeDate AS PreviousChangeDate,
    CASE 
        WHEN RPrevious.Rate IS NULL THEN NULL
        ELSE CAST(
            ((RLatest.Rate - RPrevious.Rate) * 100.0) / RPrevious.Rate
            AS DECIMAL(10, 2)
        )
    END AS PercentChange
FROM HumanResources.Employee HRE
JOIN RankedPaidHistory AS RLatest
    ON HRE.BusinessEntityID = RLatest.BusinessEntityID
    AND RLatest.rn = 1
LEFT JOIN RankedPaidHistory AS RPrevious 
    ON HRE.BusinessEntityID = RPrevious.BusinessEntityID
    AND RPrevious.rn = 2
ORDER BY PercentChange DESC;


Done.


BusinessEntityID,JobTitle,LatestRate,PreviousRate,LatestChangeDate,PreviousChangeDate,PercentChange
178,Production Technician - WC30,9.5000,7.0000,2013-07-14 00:00:00,2011-12-01 00:00:00,35.71
250,Purchasing Manager,30.0000,22.5000,2012-07-14 00:00:00,2011-07-30 00:00:00,33.33
167,Production Technician - WC30,9.5000,7.2500,2013-07-14 00:00:00,2011-12-01 00:00:00,31.03
170,Production Technician - WC30,9.5000,7.2500,2013-07-14 00:00:00,2011-12-01 00:00:00,31.03
172,Production Technician - WC30,9.5000,7.2500,2013-07-14 00:00:00,2011-12-01 00:00:00,31.03
174,Production Technician - WC30,9.5000,7.2500,2013-07-14 00:00:00,2011-12-01 00:00:00,31.03
175,Production Technician - WC30,9.5000,7.2500,2013-07-14 00:00:00,2011-12-01 00:00:00,31.03
176,Production Technician - WC30,9.5000,7.2500,2013-07-14 00:00:00,2011-12-01 00:00:00,31.03
177,Production Technician - WC30,9.5000,7.2500,2013-07-14 00:00:00,2011-12-01 00:00:00,31.03
16,Marketing Manager,37.5000,28.7500,2012-04-30 00:00:00,2009-07-15 00:00:00,30.43


Explanation

RankedPaidHistory ranks pay history per employee (most recent = rn = 1).

We join:

RLatest → latest pay record

RPrevious → previous pay record

A LEFT JOIN is used for previous rate in case an employee has only one pay record.

Percentage change is computed only when a previous rate exists.

TOPIC: CTE — Top Subcategories Discount

In this lesson, we want to identify the product subcategories with the highest average list price, and calculate a suggested 10% discount for these subcategories for high-end marketing campaigns.

Tables:

Production.Product

Concept:

CTE for aggregation and ranking

In [49]:
%%sql

WITH AvgSubcategories AS (
    SELECT
        [ProductSubcategoryID],
        AVG([ListPrice]) AS AvgListPrice
    FROM Production.Product
    GROUP BY [ProductSubcategoryID]
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    ProductSubcategoryID,
    AvgListPrice,
    AvgListPrice * 0.1 AS DiscountedAmount,
    AvgListPrice * 0.9 AS PriceAfterDiscount
FROM AvgSubcategories
ORDER BY AvgListPrice DESC;


Done.


ProductSubcategoryID,AvgListPrice,DiscountedAmount,PriceAfterDiscount
1,1683.3650,168.33650,1515.02850
2,1597.4500,159.74500,1437.70500
3,1425.2481,142.52481,1282.72329
14,780.0436,78.00436,702.03924
12,678.2535,67.82535,610.42815
16,631.4155,63.14155,568.27395
8,278.9900,27.89900,251.09100
17,220.9292,22.09292,198.83628
10,184.4000,18.44000,165.96000
27,159.0000,15.90000,143.10000


Notes

CTE (AvgSubcategories): Aggregates the average list price per subcategory.

TOP 10: Ensures the output is small in Jupyter notebooks; remove the comment to see all results.

Discount Calculations:

DiscountedAmount = AvgListPrice * 0.1

PriceAfterDiscount = AvgListPrice * 0.9

ORDER BY: Highest average price first

TOPIC: CTE — Customer Spending Above Average

In this lesson, we want to calculate each customer’s total spending and flag whether they spent more than the overall customer average.

Tables:

Sales.Customer

Sales.SalesOrderHeader

Concept:

CTE for aggregation + comparison with overall average

In [52]:
%%sql

WITH CustomersTotal AS (
    SELECT
        SC.[CustomerID],
        SC.[PersonID],
        SUM(SOH.[TotalDue]) AS TotalSpent,
        AVG(SOH.[TotalDue]) AS IndividualAVGSpent,
        (SELECT AVG(SOH2.[TotalDue]) FROM Sales.SalesOrderHeader SOH2) AS AllCustomersAvg
    FROM Sales.Customer SC
    JOIN Sales.SalesOrderHeader SOH
        ON SC.CustomerID = SOH.CustomerID
    GROUP BY SC.[CustomerID], SC.[PersonID]
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    CustomerID,
    PersonID,
    TotalSpent,
    AllCustomersAvg,
    CASE
        WHEN TotalSpent > AllCustomersAvg 
        THEN 'Above Average'
        ELSE 'Below Average'
    END AS SpendingFlag
FROM CustomersTotal
ORDER BY TotalSpent DESC;


Done.


CustomerID,PersonID,TotalSpent,AllCustomersAvg,SpendingFlag
29818,1031,989184.0820,3915.9951,Above Average
29715,813,961675.8596,3915.9951,Above Average
29722,827,954021.9235,3915.9951,Above Average
30117,1991,919801.8188,3915.9951,Above Average
29614,591,901346.8560,3915.9951,Above Average
29639,647,887090.4106,3915.9951,Above Average
29701,785,841866.5522,3915.9951,Above Average
29617,599,834475.9271,3915.9951,Above Average
29994,1417,824331.7682,3915.9951,Above Average
29646,661,820383.5466,3915.9951,Above Average


Explanation

CTE CustomersTotal

Calculates total spending per customer using SUM(TotalDue).

Computes average order value per customer (IndividualAVGSpent).

Calculates overall average spending across all customers with a scalar subquery (AllCustomersAvg).

Main Query

Selects the relevant columns from the CTE.

Uses a CASE statement to flag customers as Above Average or Below Average based on their total spending.

Orders by TotalSpent in descending order.

Limits output with TOP 10 to keep the Jupyter notebook manageable.

TOPIC: CTE — Monthly Sales Totals

In this lesson, we want to calculate the total sales amount for each month using a CTE.

Tables:

Sales.SalesOrderHeader

Concept:

CTE with GROUP BY for monthly aggregation

In [53]:
%%sql

WITH MonthlySales AS (
    SELECT 
        YEAR([OrderDate]) AS SalesYear,
        MONTH([OrderDate]) AS SalesMonth,
        SUM([TotalDue]) AS TotalSales
    FROM Sales.SalesOrderHeader
    GROUP BY YEAR([OrderDate]), MONTH([OrderDate])
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    SalesYear,
    SalesMonth,
    TotalSales
FROM MonthlySales
ORDER BY SalesYear, SalesMonth DESC;


Done.


SalesYear,SalesMonth,TotalSales
2011,12,1462448.8986
2011,11,815313.0152
2011,10,5156269.5291
2011,9,554791.6082
2011,8,2800576.1723
2011,7,2292182.8828
2011,6,507096.4690
2011,5,567020.9498
2012,12,3176848.1687
2012,11,2097153.1292


Explanation

CTE MonthlySales

Groups all sales by year and month using YEAR(OrderDate) and MONTH(OrderDate).

Calculates total sales per month with SUM(TotalDue).

Main Query

Selects SalesYear, SalesMonth, and TotalSales from the CTE.

Orders results by year ascending and month descending to see the latest months first.

Uses TOP 10 to limit output in Jupyter notebooks.

TOPIC: CTE — Products with High Inventory

In this lesson, we want to list products that have a total inventory greater than 500 across all locations.

Tables:

Production.ProductInventory

Production.Product

Concept:

CTE with SUM() and filtering by aggregate condition (HAVING)

In [54]:
%%sql

WITH InventoryAbove500 AS (
    SELECT
        [ProductID],
        SUM([Quantity]) AS TotalInventory
    FROM Production.ProductInventory
    GROUP BY [ProductID]
    HAVING SUM([Quantity]) > 500
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    PRP.[ProductID],
    PRP.[Name],
    TotalInventory
FROM Production.Product PRP
JOIN InventoryAbove500 AS PRI
    ON PRP.ProductID = PRI.ProductID
ORDER BY TotalInventory DESC;


Done.


ProductID,Name,TotalInventory
379,Hex Nut 7,1911
367,Thin-Jam Hex Nut 3,1901
383,Hex Nut 23,1901
387,Hex Nut 10,1888
393,Hex Nut 14,1880
489,Metal Tread Plate,1837
396,Hex Nut 18,1824
389,Hex Nut 2,1808
371,Thin-Jam Hex Nut 7,1781
376,Hex Nut 6,1781


Explanation

CTE InventoryAbove500

Groups inventory by ProductID.

Uses SUM(Quantity) to calculate total inventory per product.

Filters only products where total inventory > 500 using HAVING.

Main Query

Joins the CTE to the Product table to get product names.

Selects ProductID, Name, and TotalInventory.

Orders by TotalInventory descending.

Uses TOP 10 to limit results in the Jupyter notebook.

TOPIC: CTE — Territory Sales Growth

In this lesson, we want to calculate the growth rate for each sales territory by comparing SalesYTD and SalesLastYear, and rank the territories by growth rate.

Tables:

Sales.SalesTerritory

Concept:

CTE for growth calculation + ranking

In [55]:
%%sql

WITH GrowthRate AS (
    SELECT
        [TerritoryID],
        [Name],
        [SalesLastYear],
        [SalesYTD],
        CAST((([SalesYTD] - [SalesLastYear]) / [SalesLastYear]) AS DECIMAL(10, 1)) AS Rate
    FROM Sales.SalesTerritory
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    RANK() OVER (ORDER BY Rate DESC) AS GrowthRank,
    [TerritoryID],
    [Name],
    Rate
FROM GrowthRate
ORDER BY Rate DESC;


Done.


GrowthRank,TerritoryID,Name,Rate
1,10,United Kingdom,2.1
2,8,Germany,1.9
3,9,Australia,1.6
4,1,Northwest,1.4
5,4,Southwest,1.0
5,7,France,1.0
7,6,Canada,0.2
8,3,Central,0.0
9,2,Northeast,-0.3
10,5,Southeast,-0.4


Explanation

CTE GrowthRate

Calculates the growth rate per territory:

Rate
=
SalesYTD
−
SalesLastYear
SalesLastYear
Rate=
SalesLastYear
SalesYTD−SalesLastYear
	​


Casts the result as DECIMAL(10,1) for readability.

Main Query

Uses RANK() to assign a rank based on growth rate in descending order.

Selects the top 10 territories for notebook output.

Orders results by Rate to show the highest growth first.

TOPIC: CTE — Department Gender Counts

In this lesson, we want to return the total number of male and female employees per department side by side.

Tables:

HumanResources.Employee

HumanResources.EmployeeDepartmentHistory

HumanResources.Department

Concept:

Multiple CTEs (Male and Female employees) joined together for a transposed result

In [56]:
%%sql

WITH DepartmentEmployees AS (
    SELECT
        HRD.[Name] AS DepartmentName,
        HRE.[BusinessEntityID],
        HRE.[Gender]
    FROM HumanResources.Employee HRE
    JOIN HumanResources.EmployeeDepartmentHistory EDH
        ON HRE.BusinessEntityID = EDH.BusinessEntityID
    JOIN HumanResources.Department HRD
        ON EDH.DepartmentID = HRD.DepartmentID
    WHERE EDH.EndDate IS NULL
),
MaleStaffs AS (
    SELECT 
        DepartmentName,
        COUNT(*) AS MaleCount
    FROM DepartmentEmployees
    WHERE Gender = 'M'
    GROUP BY DepartmentName
),
FemaleStaffs AS (
    SELECT
        DepartmentName,
        COUNT(*) AS FemaleCount
    FROM DepartmentEmployees
    WHERE Gender = 'F'
    GROUP BY DepartmentName
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    D.DepartmentName,
    M.MaleCount,
    F.FemaleCount
FROM (SELECT DISTINCT DepartmentName FROM DepartmentEmployees) AS D
LEFT JOIN MaleStaffs AS M
    ON D.DepartmentName = M.DepartmentName
LEFT JOIN FemaleStaffs AS F
    ON D.DepartmentName = F.DepartmentName
ORDER BY D.DepartmentName;


Done.


DepartmentName,MaleCount,FemaleCount
Document Control,4,1
Engineering,3,3
Executive,1,1
Facilities and Maintenance,5,2
Finance,5,5
Human Resources,4,2
Information Services,6,4
Marketing,5,4
Production,133,46
Production Control,6,None


Explanation

CTE DepartmentEmployees

Collects all current employees (EndDate IS NULL) along with their department name and gender.

CTEs MaleStaffs and FemaleStaffs

Aggregate the count of male and female employees per department separately using COUNT(*).

Main Query

Combines male and female counts side by side by joining the CTEs on department name.

Uses DISTINCT DepartmentName to ensure all departments are included.

Limits output to TOP 10 for Jupyter notebooks.

Orders by DepartmentName for readability.

TOPIC: CTE — Department Gender Counts (Optimized)

In this lesson, we want to return the total number of male and female employees per department, but in a more efficient way using conditional aggregation.

Tables:

HumanResources.Employee

HumanResources.EmployeeDepartmentHistory

HumanResources.Department

Concept:

Single CTE + SUM with CASE for transposed counts

In [57]:
%%sql

WITH DepartmentEmployees AS (
    SELECT 
        d.Name AS DepartmentName,
        e.Gender
    FROM HumanResources.Employee e
    JOIN HumanResources.EmployeeDepartmentHistory edh
        ON e.BusinessEntityID = edh.BusinessEntityID
    JOIN HumanResources.Department d
        ON edh.DepartmentID = d.DepartmentID
    WHERE edh.EndDate IS NULL   -- only current dept assignment
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    DepartmentName,
    SUM(CASE WHEN Gender = 'M' THEN 1 ELSE 0 END) AS Male,
    SUM(CASE WHEN Gender = 'F' THEN 1 ELSE 0 END) AS Female
FROM DepartmentEmployees
GROUP BY DepartmentName
ORDER BY DepartmentName;


Done.


DepartmentName,Male,Female
Document Control,4,1
Engineering,3,3
Executive,1,1
Facilities and Maintenance,5,2
Finance,5,5
Human Resources,4,2
Information Services,6,4
Marketing,5,4
Production,133,46
Production Control,6,0


Explanation

CTE DepartmentEmployees

Selects current employees only (EndDate IS NULL) along with their department name and gender.

Main Query

Uses conditional aggregation with SUM(CASE...):

Counts males (Gender = 'M')

Counts females (Gender = 'F')

Groups by DepartmentName so each department is a single row.

Orders by department name for readability.

Limits output with TOP 10 for Jupyter notebooks.

TOPIC: CTE — Best-Selling Product per Year

In this lesson, we want to find the product that generated the highest total sales for each year.

Tables:

Sales.SalesOrderHeader

Sales.SalesOrderDetail

Production.Product

Concept:

CTE with aggregation + ROW_NUMBER() to pick the top product per year

In [58]:
%%sql

WITH SalesOrders AS (
    SELECT
        YEAR(SOH.[OrderDate]) AS SalesYear,
        PRP.[Name] AS ProductName,
        SUM(SOH.[TotalDue]) AS TotalSales
    FROM Sales.SalesOrderHeader AS SOH
    JOIN Sales.SalesOrderDetail AS SOD
        ON SOH.SalesOrderID = SOD.SalesOrderID
    JOIN Production.Product AS PRP
        ON SOD.ProductID = PRP.ProductID
    GROUP BY YEAR(SOH.[OrderDate]), PRP.[Name]
),
RankedSales AS (
    SELECT
        SalesYear,
        ProductName,
        TotalSales,
        ROW_NUMBER() OVER (PARTITION BY SalesYear ORDER BY TotalSales DESC) AS RankNumber
    FROM SalesOrders
)

SELECT TOP 10 /*remove TOP 10 to see all results*/
    SalesYear,
    ProductName,
    TotalSales
FROM RankedSales
WHERE RankNumber = 1
ORDER BY SalesYear;


Done.


SalesYear,ProductName,TotalSales
2011,AWC Logo Cap,5431991.2921
2012,AWC Logo Cap,18360243.7106
2013,AWC Logo Cap,19383256.0282
2014,Water Bottle - 30 oz.,7287883.5400


Explanation

CTE SalesOrders

Aggregates total sales (SUM(TotalDue)) per product for each year.

Groups by SalesYear and ProductName.

CTE RankedSales

Uses ROW_NUMBER() to rank products within each year based on TotalSales descending.

Rank 1 represents the highest-selling product for that year.

Main Query

Selects only products where RankNumber = 1 to get the top product per year.

Orders by SalesYear ascending.

Limits output to TOP 10 for Jupyter notebooks.